# 语言翻译

在此项目中，你将了解神经网络机器翻译这一领域。你将用由英语和法语语句组成的数据集，训练一个序列到序列模型（sequence to sequence model），该模型能够将新的英语句子翻译成法语。

## 获取数据

因为将整个英语语言内容翻译成法语需要大量训练时间，所以我们提供了一小部分的英语语料库。


In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## 探索数据

研究 view_sentence_range，查看并熟悉该数据的不同部分。


In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## 实现预处理函数

### 文本到单词 id

和之前的 RNN 一样，你必须首先将文本转换为数字，这样计算机才能读懂。在函数 `text_to_ids()` 中，你需要将单词中的 `source_text` 和 `target_text` 转为 id。但是，你需要在 `target_text` 中每个句子的末尾，添加 `<EOS>` 单词 id。这样可以帮助神经网络预测句子应该在什么地方结束。


你可以通过以下代码获取  `<EOS> ` 单词ID：

```python
target_vocab_to_int['<EOS>']
```

你可以使用 `source_vocab_to_int` 和 `target_vocab_to_int` 获得其他单词 id。


In [3]:
source_text_set = [word for line in source_text.split('\n') for word in line.split()]
print(source_text_set[:20])

['new', 'jersey', 'is', 'sometimes', 'quiet', 'during', 'autumn', ',', 'and', 'it', 'is', 'snowy', 'in', 'april', '.', 'the', 'united', 'states', 'is', 'usually']


In [4]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    #  
    eos = target_vocab_to_int['<EOS>']
    
    # source_text_set = set([word for line in source_text.split('\n') for word in line.split()])  # 不用split(' ')， 不要扁平化！！！
    source_id_text = [[source_vocab_to_int[word] for word in line.split()] for line in source_text.split('\n')]
    print(source_id_text)
    
    # target_line = [line + ' <EOS>' for line in target_text.split('\n')]
    
    target_id_text = [[target_vocab_to_int[word] for word in line.split()] + [eos] for line in target_text.split('\n')]
    print(target_id_text)
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

[[22, 11, 6, 12, 21, 15, 16, 9, 30, 14, 6, 5, 23, 18, 20], [4, 25, 29, 6, 24, 19, 15, 7, 9, 30, 14, 6, 24, 17, 23, 10, 20], [13, 6, 24, 21, 15, 26, 9, 30, 14, 6, 24, 31, 23, 27, 20], [4, 25, 29, 6, 12, 32, 15, 27, 9, 30, 14, 6, 28, 23, 8, 20]]
[[19, 11, 5, 32, 25, 33, 24, 16, 9, 4, 12, 5, 7, 29, 31, 18, 1], [17, 13, 5, 20, 27, 29, 10, 9, 4, 12, 8, 23, 29, 28, 18, 1], [14, 5, 20, 25, 29, 21, 9, 4, 12, 5, 20, 26, 29, 15, 18, 1], [17, 13, 5, 32, 22, 29, 15, 9, 4, 12, 6, 27, 29, 30, 18, 1]]
Tests Passed


### 预处理所有数据并保存

运行以下代码单元，预处理所有数据，并保存到文件中。


In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


# 检查点

这是你的第一个检查点。如果你什么时候决定再回到该记事本，或需要重新启动该记事本，可以从这里继续。预处理的数据已保存到磁盘上。

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### 检查 TensorFlow 版本，确认可访问 GPU

这一检查步骤，可以确保你使用的是正确版本的 TensorFlow，并且能够访问 GPU。


In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) in [LooseVersion('1.0.0'), LooseVersion('1.0.1')], 'This project requires TensorFlow version 1.0  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0


C:\ProgramData\Anaconda3\envs\dlndpy35\lib\site-packages\ipykernel\__main__.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.


## 构建神经网络

你将通过实现以下函数，构建出要构建一个序列到序列模型所需的组件：

- `model_inputs`
- `process_decoding_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### 输入

实现 `model_inputs()` 函数，为神经网络创建 TF 占位符。该函数应该创建以下占位符：

- 名为 “input” 的输入文本占位符，并使用 TF Placeholder 名称参数（等级（Rank）为 2）。
- 目标占位符（等级为 2）。
- 学习速率占位符（等级为 0）。
- 名为 “keep_prob” 的保留率占位符，并使用 TF Placeholder 名称参数（等级为 0）。

在以下元祖（tuple）中返回占位符：（输入、目标、学习速率、保留率）


In [8]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate, keep probability)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    return input, targets, learning_rate, keep_prob

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### 处理解码输入

使用 TensorFlow 实现 `process_decoding_input`，以便删掉 `target_data` 中每个批次的最后一个单词 ID，并将 GO ID 放到每个批次的开头。

In [9]:
def process_decoding_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for dencoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    # tf.strided_slice(data, begin, end, stride)
    end = tf.strided_slice(target_data, [0,0], [batch_size,-1], [1,1])
    decoder_input = tf.concat([tf.fill([batch_size,1], target_vocab_to_int['<GO>']), end], 1)  # 注意
    return decoder_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_decoding_input(process_decoding_input)

Tests Passed


### 编码

实现 `encoding_layer()`，以使用 [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn) 创建编码器 RNN 层级。

In [10]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :return: RNN state
    """
    # TODO: Implement Function
    cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    enc_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    _, enc_state = tf.nn.dynamic_rnn(enc_cell, rnn_inputs, dtype=tf.float32)
    return enc_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### 解码 - 训练

使用 [`tf.contrib.seq2seq.simple_decoder_fn_train()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_train) 和 [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder) 创建训练分对数（training logits）。将 `output_fn` 应用到 [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder) 输出上。

In [11]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param sequence_length: Sequence Length
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Train Logits
    """
    # TODO: Implement Function
    dec_fn_train = tf.contrib.seq2seq.simple_decoder_fn_train(encoder_state)
    outputs_train, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, dec_fn_train, dec_embed_input, sequence_length, scope=decoding_scope)
    logits_train = output_fn(outputs_train)
    logits = tf.nn.dropout(logits_train, keep_prob)
    return logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### 解码 - 推论

使用 [`tf.contrib.seq2seq.simple_decoder_fn_inference()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_inference) 和 [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder) 创建推论分对数（inference logits）。

In [12]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param maximum_length: The maximum allowed time steps to decode
    :param vocab_size: Size of vocabulary
    :param decoding_scope: TensorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Inference Logits
    """
    # TODO: Implement Function
    dec_fn_infer = tf.contrib.seq2seq.simple_decoder_fn_inference(output_fn, 
                                                                  encoder_state, 
                                                                  dec_embeddings, 
                                                                  start_of_sequence_id, 
                                                                  end_of_sequence_id, 
                                                                  maximum_length, 
                                                                  vocab_size)
    dropouted = tf.contrib.rnn.DropoutWrapper(dec_cell, keep_prob)
    logits_infer, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dropouted, dec_fn_infer, 
                                                                sequence_length=maximum_length, 
                                                                scope=decoding_scope)
    return logits_infer


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### 构建解码层级

实现 `decoding_layer()` 以创建解码器 RNN 层级。

- 使用 `rnn_size` 和 `num_layers` 创建解码 RNN 单元。
- 使用 [`lambda`](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) 创建输出函数，将输入，也就是分对数转换为类分对数（class logits）。
- 使用 `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob)` 函数获取训练分对数。
- 使用 `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size, decoding_scope, output_fn, keep_prob)` 函数获取推论分对数。

注意：你将需要使用 [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) 在训练和推论分对数间分享变量。

In [13]:
def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, target_vocab_to_int, keep_prob):
    """
    Create decoding layer
    :param dec_embed_input: Decoder embedded input
    :param dec_embeddings: Decoder embeddings
    :param encoder_state: The encoded state
    :param vocab_size: Size of vocabulary
    :param sequence_length: Sequence Length
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param keep_prob: Dropout keep probability
    :return: Tuple of (Training Logits, Inference Logits)
    """
    # TODO: Implement Function
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    dropout = tf.contrib.rnn.DropoutWrapper(lstm, keep_prob)
    dec_cell = tf.contrib.rnn.MultiRNNCell([dropout] * num_layers)
    
    max_target_sentence_length = max([len(sentence) for sentence in source_int_text])
    
    with tf.variable_scope('decoding_layer') as decoding_scope:
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, vocab_size, None, scope=decoding_scope)
    
    with tf.variable_scope('decoding_layer') as decoding_scope:
        logits_train = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                                            sequence_length, decoding_scope, output_fn, keep_prob)
    
    with tf.variable_scope('decoding_layer', reuse=True) as decoding_scope:
        logits_infer = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, 
                                            target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], 
                                            max_target_sentence_length, 
                                            vocab_size, decoding_scope, output_fn, keep_prob)
    
    return logits_train, logits_infer


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### 构建神经网络

应用你在上方实现的函数，以：

- 向编码器的输入数据应用嵌入。
- 使用 `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob)` 编码输入。
- 使用 `process_decoding_input(target_data, target_vocab_to_int, batch_size)` 函数处理目标数据。
- 向解码器的目标数据应用嵌入。
- 使用 `decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size, num_layers, target_vocab_to_int, keep_prob)` 解码编码的输入数据。

In [14]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size, rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param sequence_length: Sequence Length
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training Logits, Inference Logits)
    """
    # TODO: Implement Function
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, enc_embedding_size)
    enc_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob)
    
    dec_input = process_decoding_input(target_data, target_vocab_to_int, batch_size)
    dec_embeddings = tf.Variable(tf.truncated_normal([target_vocab_size, dec_embedding_size], stddev=0.01))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    logits_train, logits_infer = decoding_layer(dec_embed_input, dec_embeddings, 
                                                enc_state, target_vocab_size, 
                                                sequence_length, rnn_size, num_layers, 
                                                target_vocab_to_int, keep_prob)
    
    return logits_train, logits_infer


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## 训练神经网络

### 超参数

调试以下参数：

- 将 `epochs` 设为 epoch 次数。
- 将 `batch_size` 设为批次大小。
- 将 `rnn_size` 设为 RNN 的大小。
- 将 `num_layers` 设为层级数量。
- 将 `encoding_embedding_size` 设为编码器嵌入大小。
- 将 `decoding_embedding_size` 设为解码器嵌入大小
- 将 `learning_rate` 设为训练速率。
- 将 `keep_probability` 设为丢弃保留率（Dropout keep probability）。

In [15]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 3
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.01
# Dropout Keep Probability
keep_probability = 0.08

### 构建图表

使用你实现的神经网络构建图表。

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_source_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob = model_inputs()
    sequence_length = tf.placeholder_with_default(max_source_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)
    
    train_logits, inference_logits = seq2seq_model(
        tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(source_vocab_to_int), len(target_vocab_to_int),
        encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, target_vocab_to_int)

    tf.identity(inference_logits, 'logits')
    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            train_logits,
            targets,
            tf.ones([input_shape[0], sequence_length]))

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

### 训练

利用预处理的数据训练神经网络。如果很难获得低损失值，请访问我们的论坛，看看其他人是否遇到了相同的问题。

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import time

def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1]), (0,0)],
            'constant')

    return np.mean(np.equal(target, np.argmax(logits, 2)))

train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]

valid_source = helper.pad_sentence_batch(source_int_text[:batch_size])
valid_target = helper.pad_sentence_batch(target_int_text[:batch_size])

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch) in enumerate(
                helper.batch_data(train_source, train_target, batch_size)):
            start_time = time.time()
            
            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 sequence_length: target_batch.shape[1],
                 keep_prob: keep_probability})
            
            batch_train_logits = sess.run(
                inference_logits,
                {input_data: source_batch, keep_prob: 1.0})
            batch_valid_logits = sess.run(
                inference_logits,
                {input_data: valid_source, keep_prob: 1.0})
                
            train_acc = get_accuracy(target_batch, batch_train_logits)
            valid_acc = get_accuracy(np.array(valid_target), batch_valid_logits)
            end_time = time.time()
            print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.3f}, Validation Accuracy: {:>6.3f}, Loss: {:>6.3f}'
                  .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    0/538 - Train Accuracy:  0.276, Validation Accuracy:  0.353, Loss:  5.881
Epoch   0 Batch    1/538 - Train Accuracy:  0.236, Validation Accuracy:  0.319, Loss:  5.837
Epoch   0 Batch    2/538 - Train Accuracy:  0.097, Validation Accuracy:  0.177, Loss: 10.798
Epoch   0 Batch    3/538 - Train Accuracy:  0.122, Validation Accuracy:  0.207, Loss: 10.768
Epoch   0 Batch    4/538 - Train Accuracy:  0.227, Validation Accuracy:  0.307, Loss:  7.862
Epoch   0 Batch    5/538 - Train Accuracy:  0.145, Validation Accuracy:  0.182, Loss:  7.101
Epoch   0 Batch    6/538 - Train Accuracy:  0.192, Validation Accuracy:  0.238, Loss:  6.960
Epoch   0 Batch    7/538 - Train Accuracy:  0.179, Validation Accuracy:  0.253, Loss:  5.990
Epoch   0 Batch    8/538 - Train Accuracy:  0.282, Validation Accuracy:  0.351, Loss:  5.972
Epoch   0 Batch    9/538 - Train Accuracy:  0.280, Validation Accuracy:  0.349, Loss:  5.912
Epoch   0 Batch   10/538 - Train Accuracy:  0.260, Validation Accuracy

Epoch   0 Batch   89/538 - Train Accuracy:  0.175, Validation Accuracy:  0.249, Loss:  5.637
Epoch   0 Batch   90/538 - Train Accuracy:  0.213, Validation Accuracy:  0.249, Loss:  5.618
Epoch   0 Batch   91/538 - Train Accuracy:  0.181, Validation Accuracy:  0.249, Loss:  5.627
Epoch   0 Batch   92/538 - Train Accuracy:  0.177, Validation Accuracy:  0.253, Loss:  5.592
Epoch   0 Batch   93/538 - Train Accuracy:  0.188, Validation Accuracy:  0.253, Loss:  5.619
Epoch   0 Batch   94/538 - Train Accuracy:  0.183, Validation Accuracy:  0.253, Loss:  5.626
Epoch   0 Batch   95/538 - Train Accuracy:  0.256, Validation Accuracy:  0.253, Loss:  5.584
Epoch   0 Batch   96/538 - Train Accuracy:  0.218, Validation Accuracy:  0.253, Loss:  5.599
Epoch   0 Batch   97/538 - Train Accuracy:  0.184, Validation Accuracy:  0.252, Loss:  5.633
Epoch   0 Batch   98/538 - Train Accuracy:  0.222, Validation Accuracy:  0.254, Loss:  5.610
Epoch   0 Batch   99/538 - Train Accuracy:  0.187, Validation Accuracy

Epoch   0 Batch  178/538 - Train Accuracy:  0.332, Validation Accuracy:  0.361, Loss:  5.570
Epoch   0 Batch  179/538 - Train Accuracy:  0.277, Validation Accuracy:  0.339, Loss:  5.622
Epoch   0 Batch  180/538 - Train Accuracy:  0.251, Validation Accuracy:  0.281, Loss:  5.562
Epoch   0 Batch  181/538 - Train Accuracy:  0.204, Validation Accuracy:  0.268, Loss:  5.589
Epoch   0 Batch  182/538 - Train Accuracy:  0.206, Validation Accuracy:  0.267, Loss:  5.594
Epoch   0 Batch  183/538 - Train Accuracy:  0.269, Validation Accuracy:  0.294, Loss:  5.576
Epoch   0 Batch  184/538 - Train Accuracy:  0.317, Validation Accuracy:  0.347, Loss:  5.589
Epoch   0 Batch  185/538 - Train Accuracy:  0.304, Validation Accuracy:  0.365, Loss:  5.593
Epoch   0 Batch  186/538 - Train Accuracy:  0.348, Validation Accuracy:  0.369, Loss:  5.606
Epoch   0 Batch  187/538 - Train Accuracy:  0.369, Validation Accuracy:  0.365, Loss:  5.592
Epoch   0 Batch  188/538 - Train Accuracy:  0.295, Validation Accuracy

Epoch   0 Batch  267/538 - Train Accuracy:  0.322, Validation Accuracy:  0.390, Loss:  5.593
Epoch   0 Batch  268/538 - Train Accuracy:  0.359, Validation Accuracy:  0.387, Loss:  5.574
Epoch   0 Batch  269/538 - Train Accuracy:  0.321, Validation Accuracy:  0.392, Loss:  5.609
Epoch   0 Batch  270/538 - Train Accuracy:  0.339, Validation Accuracy:  0.397, Loss:  5.615
Epoch   0 Batch  271/538 - Train Accuracy:  0.343, Validation Accuracy:  0.384, Loss:  5.607
Epoch   0 Batch  272/538 - Train Accuracy:  0.311, Validation Accuracy:  0.388, Loss:  5.596
Epoch   0 Batch  273/538 - Train Accuracy:  0.342, Validation Accuracy:  0.394, Loss:  5.614
Epoch   0 Batch  274/538 - Train Accuracy:  0.323, Validation Accuracy:  0.386, Loss:  5.608
Epoch   0 Batch  275/538 - Train Accuracy:  0.329, Validation Accuracy:  0.386, Loss:  5.574
Epoch   0 Batch  276/538 - Train Accuracy:  0.319, Validation Accuracy:  0.381, Loss:  5.565
Epoch   0 Batch  277/538 - Train Accuracy:  0.312, Validation Accuracy

Epoch   0 Batch  356/538 - Train Accuracy:  0.386, Validation Accuracy:  0.388, Loss:  5.563
Epoch   0 Batch  357/538 - Train Accuracy:  0.334, Validation Accuracy:  0.388, Loss:  5.572
Epoch   0 Batch  358/538 - Train Accuracy:  0.324, Validation Accuracy:  0.381, Loss:  5.603
Epoch   0 Batch  359/538 - Train Accuracy:  0.356, Validation Accuracy:  0.386, Loss:  5.553
Epoch   0 Batch  360/538 - Train Accuracy:  0.327, Validation Accuracy:  0.388, Loss:  5.592
Epoch   0 Batch  361/538 - Train Accuracy:  0.368, Validation Accuracy:  0.394, Loss:  5.599
Epoch   0 Batch  362/538 - Train Accuracy:  0.366, Validation Accuracy:  0.399, Loss:  5.573
Epoch   0 Batch  363/538 - Train Accuracy:  0.363, Validation Accuracy:  0.396, Loss:  5.555
Epoch   0 Batch  364/538 - Train Accuracy:  0.322, Validation Accuracy:  0.400, Loss:  5.592
Epoch   0 Batch  365/538 - Train Accuracy:  0.375, Validation Accuracy:  0.415, Loss:  5.573
Epoch   0 Batch  366/538 - Train Accuracy:  0.358, Validation Accuracy

...

Epoch   3 Batch  436/538 - Train Accuracy:  0.269, Validation Accuracy:  0.345, Loss:  5.504
Epoch   3 Batch  437/538 - Train Accuracy:  0.276, Validation Accuracy:  0.342, Loss:  5.534
Epoch   3 Batch  438/538 - Train Accuracy:  0.315, Validation Accuracy:  0.380, Loss:  5.519
Epoch   3 Batch  439/538 - Train Accuracy:  0.315, Validation Accuracy:  0.380, Loss:  5.488
Epoch   3 Batch  440/538 - Train Accuracy:  0.291, Validation Accuracy:  0.361, Loss:  5.495
Epoch   3 Batch  441/538 - Train Accuracy:  0.291, Validation Accuracy:  0.364, Loss:  5.486
Epoch   3 Batch  442/538 - Train Accuracy:  0.355, Validation Accuracy:  0.364, Loss:  5.512
Epoch   3 Batch  443/538 - Train Accuracy:  0.312, Validation Accuracy:  0.359, Loss:  5.487
Epoch   3 Batch  444/538 - Train Accuracy:  0.327, Validation Accuracy:  0.358, Loss:  5.469
Epoch   3 Batch  445/538 - Train Accuracy:  0.303, Validation Accuracy:  0.376, Loss:  5.524
Epoch   3 Batch  446/538 - Train Accuracy:  0.357, Validation Accuracy

Epoch   3 Batch  525/538 - Train Accuracy:  0.451, Validation Accuracy:  0.475, Loss:  5.489
Epoch   3 Batch  526/538 - Train Accuracy:  0.452, Validation Accuracy:  0.476, Loss:  5.486
Epoch   3 Batch  527/538 - Train Accuracy:  0.433, Validation Accuracy:  0.477, Loss:  5.512
Epoch   3 Batch  528/538 - Train Accuracy:  0.388, Validation Accuracy:  0.481, Loss:  5.509
Epoch   3 Batch  529/538 - Train Accuracy:  0.420, Validation Accuracy:  0.481, Loss:  5.506
Epoch   3 Batch  530/538 - Train Accuracy:  0.394, Validation Accuracy:  0.469, Loss:  5.499
Epoch   3 Batch  531/538 - Train Accuracy:  0.345, Validation Accuracy:  0.417, Loss:  5.517
Epoch   3 Batch  532/538 - Train Accuracy:  0.341, Validation Accuracy:  0.414, Loss:  5.467
Epoch   3 Batch  533/538 - Train Accuracy:  0.352, Validation Accuracy:  0.412, Loss:  5.490
Epoch   3 Batch  534/538 - Train Accuracy:  0.364, Validation Accuracy:  0.411, Loss:  5.502
Epoch   3 Batch  535/538 - Train Accuracy:  0.331, Validation Accuracy

Epoch   4 Batch   77/538 - Train Accuracy:  0.310, Validation Accuracy:  0.387, Loss:  5.503
Epoch   4 Batch   78/538 - Train Accuracy:  0.339, Validation Accuracy:  0.385, Loss:  5.490
Epoch   4 Batch   79/538 - Train Accuracy:  0.338, Validation Accuracy:  0.387, Loss:  5.452
Epoch   4 Batch   80/538 - Train Accuracy:  0.302, Validation Accuracy:  0.387, Loss:  5.517
Epoch   4 Batch   81/538 - Train Accuracy:  0.304, Validation Accuracy:  0.387, Loss:  5.501
Epoch   4 Batch   82/538 - Train Accuracy:  0.307, Validation Accuracy:  0.385, Loss:  5.506
Epoch   4 Batch   83/538 - Train Accuracy:  0.296, Validation Accuracy:  0.376, Loss:  5.492
Epoch   4 Batch   84/538 - Train Accuracy:  0.334, Validation Accuracy:  0.376, Loss:  5.455
Epoch   4 Batch   85/538 - Train Accuracy:  0.347, Validation Accuracy:  0.363, Loss:  5.490
Epoch   4 Batch   86/538 - Train Accuracy:  0.285, Validation Accuracy:  0.372, Loss:  5.522
Epoch   4 Batch   87/538 - Train Accuracy:  0.320, Validation Accuracy

Epoch   4 Batch  166/538 - Train Accuracy:  0.276, Validation Accuracy:  0.329, Loss:  5.517
Epoch   4 Batch  167/538 - Train Accuracy:  0.358, Validation Accuracy:  0.383, Loss:  5.485
Epoch   4 Batch  168/538 - Train Accuracy:  0.303, Validation Accuracy:  0.402, Loss:  5.496
Epoch   4 Batch  169/538 - Train Accuracy:  0.416, Validation Accuracy:  0.472, Loss:  5.490
Epoch   4 Batch  170/538 - Train Accuracy:  0.433, Validation Accuracy:  0.472, Loss:  5.493
Epoch   4 Batch  171/538 - Train Accuracy:  0.420, Validation Accuracy:  0.474, Loss:  5.506
Epoch   4 Batch  172/538 - Train Accuracy:  0.362, Validation Accuracy:  0.408, Loss:  5.509
Epoch   4 Batch  173/538 - Train Accuracy:  0.378, Validation Accuracy:  0.407, Loss:  5.500
Epoch   4 Batch  174/538 - Train Accuracy:  0.344, Validation Accuracy:  0.408, Loss:  5.513
Epoch   4 Batch  175/538 - Train Accuracy:  0.291, Validation Accuracy:  0.397, Loss:  5.496
Epoch   4 Batch  176/538 - Train Accuracy:  0.317, Validation Accuracy

Epoch   4 Batch  255/538 - Train Accuracy:  0.243, Validation Accuracy:  0.313, Loss:  5.495
Epoch   4 Batch  256/538 - Train Accuracy:  0.247, Validation Accuracy:  0.314, Loss:  5.506
Epoch   4 Batch  257/538 - Train Accuracy:  0.295, Validation Accuracy:  0.315, Loss:  5.511
Epoch   4 Batch  258/538 - Train Accuracy:  0.290, Validation Accuracy:  0.311, Loss:  5.506
Epoch   4 Batch  259/538 - Train Accuracy:  0.283, Validation Accuracy:  0.307, Loss:  5.475
Epoch   4 Batch  260/538 - Train Accuracy:  0.305, Validation Accuracy:  0.334, Loss:  5.476
Epoch   4 Batch  261/538 - Train Accuracy:  0.308, Validation Accuracy:  0.383, Loss:  5.484
Epoch   4 Batch  262/538 - Train Accuracy:  0.315, Validation Accuracy:  0.384, Loss:  5.506
Epoch   4 Batch  263/538 - Train Accuracy:  0.331, Validation Accuracy:  0.384, Loss:  5.491
Epoch   4 Batch  264/538 - Train Accuracy:  0.420, Validation Accuracy:  0.472, Loss:  5.495
Epoch   4 Batch  265/538 - Train Accuracy:  0.413, Validation Accuracy

### 保存参数

保存 `batch_size` 和 `save_path` 参数以进行推论（for inference）。

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# 检查点

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## 句子到序列

要向模型提供要翻译的句子，你首先需要预处理该句子。实现函数 `sentence_to_seq()` 以预处理新的句子。

- 将句子转换为小写形式
- 使用 `vocab_to_int` 将单词转换为 id
 - 如果单词不在词汇表中，将其转换为`<UNK>` 单词 id

In [ ]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    return None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

## 翻译

将 `translate_sentence` 从英语翻译成法语。

In [ ]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('logits:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence], keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in np.argmax(translate_logits, 1)]))
print('  French Words: {}'.format([target_int_to_vocab[i] for i in np.argmax(translate_logits, 1)]))

## 不完美的翻译

你可能注意到了，某些句子的翻译质量比其他的要好。因为你使用的数据集只有 227 个英语单词，但实际生活中有数千个单词，只有使用这些单词的句子结果才会比较理想。对于此项目，不需要达到完美的翻译。但是，如果你想创建更好的翻译模型，则需要更好的数据。

你可以使用 [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar) 语料库训练模型。该数据集拥有更多的词汇，讨论的话题也更丰富。但是，训练时间要好多天的时间，所以确保你有 GPU 并且对于我们提供的数据集，你的神经网络性能很棒。提交此项目后，别忘了研究下 WMT10 语料库。


## 提交项目

提交项目时，确保先运行所有单元，然后再保存记事本。保存记事本文件为 “dlnd_language_translation.ipynb”，再通过菜单中的“文件” ->“下载为”将其另存为 HTML 格式。提交的项目文档中需包含“helper.py”和“problem_unittests.py”文件。
